# 交易日历

In [ ]:
from trading_calendars import get_calendar

In [ ]:
calendar = get_calendar('XSHG')

In [ ]:
calendar.schedule.index

In [ ]:
calendar.schedule.tail()

# 模拟期间

In [ ]:
import pandas as pd

In [ ]:
sessions = pd.date_range('2020-06-20', '2020-06-30', tz='UTC')

In [ ]:
trading_sessions = calendar.schedule.index.intersection(sessions)
trading_o_and_c = calendar.schedule.loc[trading_sessions]

In [ ]:
trading_sessions

In [ ]:
trading_o_and_c

In [ ]:
from zipline.algorithm import *

# 模拟时钟

In [ ]:
market_closes = trading_o_and_c['market_close']
minutely_emission = False

In [ ]:
execution_closes = calendar.execution_time_from_close(market_closes)
execution_opens = execution_closes

In [ ]:
before_trading_start_minutes = days_at_time(
    trading_sessions,
    time(9, 30),  # √
    "Asia/Shanghai"  # √
)

In [ ]:
STATUS = {
    0: 'BAR',
    1: 'SESSION_START',
    2: 'SESSION_END',
    3: 'MINUTE_END',
    4: 'BEFORE_TRADING_START_BAR'
}

## `daily mode`
in daily mode, we want to have one bar per session, timestamped as the last minute of the session.

In [ ]:
clock = MinuteSimulationClock(
    trading_sessions,
    execution_opens,
    execution_closes,
    before_trading_start_minutes,
    minute_emission=False,
)

In [ ]:
for dt, ac in clock:
    print(dt.tz_convert('Asia/Shanghai'), STATUS[ac])

## 考虑午休的分钟级别模式

In [ ]:
market_opens = trading_o_and_c['market_open']
execution_opens = calendar.execution_time_from_open(market_opens)
execution_closes = calendar.execution_time_from_close(market_closes)

In [ ]:
clock = MinuteSimulationClock(
    trading_sessions,
    execution_opens,
    execution_closes,
    before_trading_start_minutes,
    minute_emission=True,
)

In [ ]:
for dt, ac in clock:
    print(dt.tz_convert('Asia/Shanghai'), STATUS[ac])